<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/html_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
# Import packages

import os
import json
import re
from bs4 import BeautifulSoup
import requests

In [9]:
# Global parameters

URL_bipolar = 'https://www.nice.org.uk/guidance/cg185/chapter/Recommendations'
URL_abortion = 'https://www.nice.org.uk/guidance/ng140/chapter/Recommendations'
URL_pneumonia = 'https://www.nice.org.uk/guidance/ng250/chapter/Recommendations'
URL_schizophrenia = 'https://www.nice.org.uk/guidance/cg178/chapter/Recommendations'
URL_borderline_pd = 'https://www.nice.org.uk/guidance/cg78/chapter/Recommendations'
URL_alcohol_use_disorders = 'https://www.nice.org.uk/guidance/cg115/chapter/Recommendations'
URL_depression_in_adults = 'https://www.nice.org.uk/guidance/ng222/chapter/Recommendations'
URL_self_harm = 'https://www.nice.org.uk/guidance/ng225/chapter/Recommendations'
URL_social_anxiety = 'https://www.nice.org.uk/guidance/cg159/chapter/Recommendations'
URL_ocd = 'https://www.nice.org.uk/guidance/cg31/chapter/Recommendations'
URL_eating_disorders = 'https://www.nice.org.uk/guidance/ng69/chapter/Recommendations'
URL_ptsd = 'https://www.nice.org.uk/guidance/ng116/chapter/Recommendations'
URL_gad_and_panic = 'https://www.nice.org.uk/guidance/cg113/chapter/Recommendations'

SAVE_PATH = '/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes'
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_raw.json")

In [ ]:
# Scrape the text

url = URL_bipolar

html_scrape = requests.get(url).text

soup = BeautifulSoup(html_scrape, "html.parser")

print(soup.prettify())

# Save scrape with metadata?

In [18]:
# Processing function
# See https://github.com/awsdataarchitect/medium2dev/blob/main/medium2dev.py
# See https://cozy.computer/scraping-speculative-fiction-short-stories-using-python-and-beautiful-soup-4

def process_html(soup):

    raw_recommendations = []

    # Find start point where recommendation headings begin
    start = soup.find("h3", string=lambda s: s and s.strip().startswith("1.1"))
    if start is None:
        return raw_recommendations

    # For cleaning whitespace and removing text artefacts
    def clean(s):
        return " ".join(s.replace("\xa0", " ").replace("[2014]", " ").split()).strip()

    # Capture first h3 before iterating
    raw_recommendations.append(f"HEADING: {clean(start.get_text(' ', strip=True))}")

    # Iterate through elements by name and class
    for el in start.next_elements:
        if getattr(el, "name", None) is None:
            continue

        classes = el.get("class", [])
        text = clean(el.get_text(" ", strip=True))
        if not text:
            continue

        if el.name == "h3" and "title" in classes:
            raw_recommendations.append(f"HEADING: {text}")

        elif el.name == "h4" and "title" in classes:
            raw_recommendations.append(f"SUB_HEADING_1: {text}")

        elif el.name == "h5" and "title" in classes:
            raw_recommendations.append(f"SUB_HEADING_2: {text}")

        elif el.name in ("h4", "h5", "h6") and "recommendation__number" in classes:
            raw_recommendations.append(f"REC_NUM: {text}")

        elif el.name == "div" and "recommendation__body" in classes:
            p = el.find("p")
            if p:
                raw_recommendations.append(f"REC_TEXT: {clean(p.get_text(' ', strip=True))}")

        elif el.name == "strong":
            raw_recommendations.append(f"PANEL: {text}")

        elif el.name == "li":
            raw_recommendations.append(f"BULLET: {text}")

    return raw_recommendations

In [19]:
raw_guideline_data = process_html(soup)

print(raw_guideline_data)

json.dump(raw_guideline_data, open(SAVE_FILE.replace(".txt", ".json"), "w", encoding="utf-8"), ensure_ascii=False, indent=2)

# Save guideline data with metadata ?

# Include format validation?

['HEADING: 1.1 Care for adults, children and young people across all phases of bipolar disorder', 'SUB_HEADING_1: Treatment and support for specific populations', 'REC_NUM: 1.1.1', 'REC_TEXT: Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.', 'REC_NUM: 1.1.2', "REC_TEXT: Offer people with bipolar disorder and coexisting disorders, such as personality disorder, attention deficit hyperactivity disorder, anxiety disorders or substance misuse, treatment in line with the relevant NICE guideline, in addition to their treatment for bipolar disorder. See NICE's guidelines on antisocial personality disorder , borderline personality disorder , attention deficit hyperactivity disorder , generalised anxiety disorder and panic disorder in adults and psychosis with coexisting substance misuse , be alert to the potential for drug interactions and use clinical judgement.", 'REC_NUM: 1.1.3', 'REC_TEXT: Offer pe